Dan Shea  
2017.07.24  

`get_FPKM_data` takes the overlap files computed by `bedtools intersect` and the FPKM expression calculated by `cuffdiff` and creates tables of NATs and incRNAs paired with their respective overlapping genes.

In [1]:
import pandas as pd
import re

In [2]:
NAT_overlap_file    = 'NAT_gene_overlaps.txt'
incRNA_overlap_file = 'incRNA_gene_overlaps.txt'
expression_file     = 'FPKM_data.csv'

In [3]:
gene_exp = pd.read_table(expression_file)
incRNA_pairs = pd.read_table(incRNA_overlap_file, header=None)
NAT_pairs = pd.read_table(NAT_overlap_file, header=None)

In [4]:
gene_exp

,test_id,NV_FPKM,V_FPKM,q_value,significant,len,Class
0,MSTRG.10000,0.991745,2.126390,0.412578,no,308,putative mRNA
1,MSTRG.10022,7.532030,6.144130,0.276294,no,6330,putative mRNA
2,MSTRG.10078,4.492290,4.700060,0.862383,no,2651,putative mRNA
3,MSTRG.10087,6.724960,15.236800,0.006994,yes,1201,putative mRNA
4,MSTRG.10136,242.735000,162.915000,0.038957,yes,4882,putative mRNA
5,MSTRG.10137,4.740240,3.838070,0.916878,no,354,putative mRNA
6,MSTRG.10197,20.328400,16.673200,0.746339,no,715,putative mRNA
7,MSTRG.10279,4.590450,3.295220,0.220994,no,1376,putative mRNA
8,MSTRG.10365,29.237000,37.350500,0.257047,no,567,putative mRNA
9,MSTRG.10696,87.962000,91.137500,0.877985,no,502,putative mRNA


In [5]:
incRNA_pairs

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,A01,blat,gene,1740077,1744516,1.00,-,.,ID=Bra011545;geneID=Bra011545;gene_name=Bra011545,A01,StringTie,transcript,1740730,1741066,1000,-,.,"gene_id ""MSTRG.259""; transcript_id ""MSTRG.259...."
1,A01,blat,gene,5315328,5324850,1.00,-,.,ID=Bra013383;geneID=Bra013383;gene_name=Bra013383,A01,StringTie,transcript,5320477,5322004,1000,-,.,"gene_id ""MSTRG.742""; transcript_id ""MSTRG.742...."
2,A01,glean,gene,6927844,6934400,0.74,+,.,ID=Bra013661;geneID=Bra013661;gene_name=Bra013661,A01,StringTie,transcript,6930860,6932128,1000,+,.,"gene_id ""MSTRG.933""; transcript_id ""MSTRG.933...."
3,A01,glean,gene,7104831,7119006,0.62,-,.,ID=Bra013682;geneID=Bra013682;gene_name=Bra013682,A01,StringTie,transcript,7108842,7109819,1000,-,.,"gene_id ""MSTRG.966""; transcript_id ""MSTRG.966...."
4,A01,blat,gene,9787046,9789344,1.00,-,.,ID=Bra026326;geneID=Bra026326;gene_name=Bra026326,A01,StringTie,transcript,9788268,9788607,1000,-,.,"gene_id ""MSTRG.1240""; transcript_id ""MSTRG.124..."
5,A01,blat,gene,17355509,17366323,1.00,+,.,ID=Bra031431;geneID=Bra031431;gene_name=Bra031431,A01,StringTie,transcript,17364715,17365182,1000,+,.,"gene_id ""MSTRG.1734""; transcript_id ""MSTRG.173..."
6,A01,glean,gene,19514143,19518454,0.96,+,.,ID=Bra023842;geneID=Bra023842;gene_name=Bra023842,A01,StringTie,transcript,19516164,19517466,1000,+,.,"gene_id ""MSTRG.1863""; transcript_id ""MSTRG.186..."
7,A01,glean,gene,19832553,19842484,0.98,+,.,ID=Bra023873;geneID=Bra023873;gene_name=Bra023873,A01,StringTie,transcript,19839184,19839540,1000,+,.,"gene_id ""MSTRG.1884""; transcript_id ""MSTRG.188..."
8,A02,glean,gene,873674,883648,0.90,-,.,ID=Bra028746;geneID=Bra028746;gene_name=Bra028746,A02,StringTie,transcript,876219,877653,1000,-,.,"gene_id ""MSTRG.2678""; transcript_id ""MSTRG.267..."
9,A02,glean,gene,2984626,2996249,0.57,+,.,ID=Bra023578;geneID=Bra023578;gene_name=Bra023578,A02,StringTie,transcript,2993020,2994079,1000,+,.,"gene_id ""MSTRG.2974""; transcript_id ""MSTRG.297..."


In [6]:
NAT_pairs

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,A01,blat,gene,8426,10051,1.00,-,.,ID=Bra011902;geneID=Bra011902;gene_name=Bra011902,A01,StringTie,transcript,7593,8521,1000,+,.,"gene_id ""MSTRG.6""; transcript_id ""MSTRG.6.1"";"
1,A01,blat,gene,631422,632295,1.00,+,.,ID=Bra011772;geneID=Bra011772;gene_name=Bra011772,A01,StringTie,transcript,631715,632355,1000,-,.,"gene_id ""MSTRG.86""; transcript_id ""MSTRG.86.1"";"
2,A01,blat,gene,726038,726762,1.00,-,.,ID=Bra011754;geneID=Bra011754;gene_name=Bra011754,A01,StringTie,transcript,725978,726239,1000,+,.,"gene_id ""MSTRG.99""; transcript_id ""MSTRG.99.1"";"
3,A01,blat,gene,944589,945680,1.00,-,.,ID=Bra011709;geneID=Bra011709;gene_name=Bra011709,A01,StringTie,transcript,944323,946019,1000,+,.,"gene_id ""MSTRG.127""; transcript_id ""MSTRG.127...."
4,A01,blat,gene,1059150,1060552,1.00,-,.,ID=Bra011685;geneID=Bra011685;gene_name=Bra011685,A01,StringTie,transcript,1059088,1060762,1000,+,.,"gene_id ""MSTRG.147""; transcript_id ""MSTRG.147...."
5,A01,blat,gene,1247218,1247693,1.00,+,.,ID=Bra011652;geneID=Bra011652;gene_name=Bra011652,A01,StringTie,transcript,1247674,1249420,1000,-,.,"gene_id ""MSTRG.178""; transcript_id ""MSTRG.178...."
6,A01,blat,gene,1333415,1333663,1.00,-,.,ID=Bra011634;geneID=Bra011634;gene_name=Bra011634,A01,StringTie,transcript,1333206,1333861,1000,+,.,"gene_id ""MSTRG.189""; transcript_id ""MSTRG.189...."
7,A01,blat,gene,1698882,1704489,1.00,+,.,ID=Bra011555;geneID=Bra011555;gene_name=Bra011555,A01,StringTie,transcript,1703433,1704492,1000,-,.,"gene_id ""MSTRG.248""; transcript_id ""MSTRG.248...."
8,A01,blat,gene,2185269,2188581,1.00,+,.,ID=Bra011433;geneID=Bra011433;gene_name=Bra011433,A01,StringTie,transcript,2188022,2189892,1000,-,.,"gene_id ""MSTRG.335""; transcript_id ""MSTRG.335...."
9,A01,blat,gene,3445591,3446097,1.00,-,.,ID=Bra011183;geneID=Bra011183;gene_name=Bra011183,A01,StringTie,transcript,3445540,3446158,1000,+,.,"gene_id ""MSTRG.516""; transcript_id ""MSTRG.516...."


In [7]:
# parse the gene names for genes that overlap incRNAs
gene_list = list()
for entry in incRNA_pairs.iloc[:,8]:
    gene_list.append((entry.split(';')[2]).split('=')[1])
incRNA_pairs.iloc[:,8] = gene_list

In [8]:
# parse the transcript ids for incRNAs
incRNA_list = list()
for entry in incRNA_pairs.iloc[:,17]:
    incRNA_list.append(((entry.split(';')[0]).split(' ')[1]).strip('\"'))
incRNA_pairs.iloc[:,17] = incRNA_list

In [9]:
# parse the gene names for genes that overlap to NATs
gene_list = list()
for entry in NAT_pairs.iloc[:,8]:
    gene_list.append((entry.split(';')[2]).split('=')[1])
NAT_pairs.iloc[:,8] = gene_list

In [10]:
# parse the transcript ids for NATs
NAT_list = list()
for entry in NAT_pairs.iloc[:,17]:
    NAT_list.append(((entry.split(';')[0]).split(' ')[1]).strip('\"'))
NAT_pairs.iloc[:,17] = NAT_list

In [11]:
# get the expression information for genes that overlap incRNAs
incRNA_gene_exp = gene_exp.loc[gene_exp.test_id.isin(incRNA_pairs.iloc[:,8]) , ['test_id', 'NV_FPKM', 'V_FPKM']]

In [12]:
# get the expression information for incRNAs
incRNA_incRNA_exp = gene_exp.loc[gene_exp.test_id.isin(incRNA_pairs.iloc[:,17]) , ['test_id', 'NV_FPKM', 'V_FPKM']]

In [13]:
# Rename the columns of the data frames
incRNA_gene_exp = incRNA_gene_exp.rename(columns = {'test_id':'gene', 'NV_FPKM':'gene_NV_FPKM', 'V_FPKM':'gene_V_FPKM'})
incRNA_incRNA_exp = incRNA_incRNA_exp.rename(columns = {'test_id':'incRNA', 'NV_FPKM':'incRNA_NV_FPKM', 'V_FPKM':'incRNA_V_FPKM'})

In [14]:
# get the expression information for genes that overlap NATs
NAT_gene_exp = gene_exp.loc[gene_exp.test_id.isin(NAT_pairs.iloc[:,8]) , ['test_id', 'NV_FPKM', 'V_FPKM']]

In [15]:
# get the expression information for NATs
NAT_NAT_exp = gene_exp.loc[gene_exp.test_id.isin(NAT_pairs.iloc[:,17]) , ['test_id', 'NV_FPKM', 'NV_FPKM']]

In [16]:
# Rename the columns of the data frames
NAT_gene_exp = NAT_gene_exp.rename(columns = {'test_id':'gene', 'NV_FPKM':'gene_NV_FPKM', 'V_FPKM':'gene_V_FPKM'})
NAT_NAT_exp = NAT_NAT_exp.rename(columns = {'test_id':'NAT', 'NV_FPKM':'NAT_NV_FPKM', 'V_FPKM':'NAT_V_FPKM'})

In [17]:
# Genes that don't have an overlapping incRNA
gene_no_incRNA = gene_exp.loc[~gene_exp.test_id.isin(incRNA_pairs.iloc[:,8]), ['test_id', 'NV_FPKM', 'V_FPKM']]
gene_no_incRNA = gene_no_incRNA.loc[gene_no_incRNA.test_id.str.startswith('Bra'), :]
gene_no_incRNA = gene_no_incRNA.rename(columns = {'test_id':'gene', 'NV_FPKM':'gene_NV_FPKM', 'V_FPKM':'gene_V_FPKM'})

In [18]:
# Genes that don't have an overlapping NAT
gene_no_NAT = gene_exp.loc[~gene_exp.test_id.isin(NAT_pairs.iloc[:,8]) , ['test_id', 'NV_FPKM', 'V_FPKM']]
gene_no_NAT = gene_no_NAT.loc[gene_no_NAT.test_id.str.startswith('Bra'), :]
gene_no_NAT = gene_no_NAT.rename(columns = {'test_id':'gene', 'NV_FPKM':'gene_NV_FPKM', 'V_FPKM':'gene_V_FPKM'})

In [19]:
import statistics

In [20]:
statistics.mean(incRNA_gene_exp.gene_NV_FPKM)

5.8000675416666665

In [21]:
statistics.mean(incRNA_gene_exp.gene_V_FPKM)

5.7302958333333338

In [22]:
statistics.mean(gene_no_incRNA.gene_NV_FPKM)

44.973238345886038

In [23]:
statistics.mean(gene_no_incRNA.gene_V_FPKM)

35.714600837529971

In [24]:
statistics.mean(NAT_gene_exp.gene_NV_FPKM)

75.755004825520828

In [25]:
statistics.mean(NAT_gene_exp.gene_V_FPKM)

69.516425573437502

In [26]:
statistics.mean(gene_no_NAT.gene_NV_FPKM)

44.439577920664128

In [27]:
statistics.mean(gene_no_NAT.gene_V_FPKM)

35.141662691125006

In [28]:
incRNA_gene_exp.shape

(24, 3)

In [29]:
NAT_gene_exp.shape

(384, 3)

In [30]:
gene_no_incRNA.shape

(24271, 3)

In [31]:
gene_no_NAT.shape

(23911, 3)